# Mini-project for D7041E
Alex Bergdahl - alxber-0@student.ltu.se<br/>
Linus Håkegård - linhke-0@student.ltu.se

## Import dataset

In [158]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim

## Train the models

### Set up the training and testing data

In [159]:
batch_size_train = 64
batch_size_test = 1000

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size_train, shuffle=True)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size_test, shuffle=False)

### Define the neural network model

In [160]:
class Net(nn.Module):
    def __init__(self, num_hidden_layers, hidden_layer_sizes):
        super(Net, self).__init__()
        self.num_hidden_layers = num_hidden_layers
        self.hidden_layers = nn.ModuleList()

        # Input layer
        self.hidden_layers.append(nn.Linear(28 * 28, hidden_layer_sizes[0]))

        # Hidden layers
        for i in range(num_hidden_layers - 1):
            self.hidden_layers.append(nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1]))

        # Output layer
        self.output_layer = nn.Linear(hidden_layer_sizes[-1], 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        
        # Pass through hidden layers
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        
        # Output layer
        x = self.output_layer(x)
        return x

### Define the training function

In [161]:
def train_model(criterion, epochs, num_layers_list, layer_sizes_list):
    models = []
    
    for num_hidden_layers, hidden_layer_sizes in zip(num_layers_list, layer_sizes_list):
        print(f'Number of hidden layers: {num_hidden_layers}, Hidden layer sizes: {hidden_layer_sizes}')
        for hidden_layer_size in hidden_layer_sizes:
            net = Net(num_hidden_layers, hidden_layer_size)
            optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

            for epoch in range(epochs):
                running_loss = 0.0
                for i, data in enumerate(trainloader, 0):
                    inputs, labels = data
                    optimizer.zero_grad()
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    running_loss += loss.item()
                print(f'Epoch {epoch + 1}, Loss: {running_loss / len(trainloader)}')
            
            # print(f'Number of hidden layers: {num_hidden_layers}, Hidden layer sizes: {hidden_layer_size}')
            # print(type(criterion).__name__)
            # torch.save(net.state_dict(), f'./models/{type(criterion).__name__}_{num_hidden_layers}_{hidden_layer_size}_model_weights.pth')
            models.append((net, num_hidden_layers, hidden_layer_size))
    return models

### Train the model using cross entropy cost function

In [162]:
criterion = nn.CrossEntropyLoss()
epochs = 6
num_layers_list = [1]
# layer_sizes_list = [[(256,), (512,), (1024,)], [(256, 256), (512, 512), (1024, 1024)], [(256, 256, 256), (512, 512, 512), (1024, 1024, 1024)]]
layer_sizes_list = [[(256,)]]

# models_CrossEntropy = train_model(criterion, 2, [1, 2, 3], layer_sizes_list)

criterion = nn.AbsCriterion()

models_L1Loss = train_model(criterion, epochs, [1, 2, 3], layer_sizes_list)



AttributeError: module 'torch.nn' has no attribute 'AbsCriterion'

## Test the models